In [3]:
import torch
import torch.nn as nn

# Define the convolutional layer and layer normalization
conv = nn.Conv2d(in_channels=3, out_channels=16,
                 kernel_size=3, stride=1, padding=1)
# Assume the output size is [16, 32, 32]
layer_norm = nn.LayerNorm([16, 32, 32])

# Example input
x = torch.randn(1, 3, 32, 32)

# Forward pass through the original layers
conv_output = conv(x)
original_output = layer_norm(conv_output)

# Extract parameters
W = conv.weight.data
b = conv.bias.data
gamma = layer_norm.weight.data
beta = layer_norm.bias.data
epsilon = layer_norm.eps

# Compute mean and variance of the convolutional output
mu = torch.mean(conv_output, dim=(0, 2, 3), keepdim=True)
sigma2 = torch.var(conv_output, dim=(0, 2, 3), keepdim=True, unbiased=False)

# Compute new weights and biases
W_prime = gamma.view(-1, 1, 1, 1) * W / torch.sqrt(sigma2 + epsilon)
b_prime = (b - mu.squeeze()) / \
    torch.sqrt(sigma2.squeeze() + epsilon) * gamma + beta

# Create a new convolutional layer with the merged parameters
merged_conv = nn.Conv2d(in_channels=3, out_channels=16,
                        kernel_size=3, stride=1, padding=1)
merged_conv.weight.data = W_prime
merged_conv.bias.data = b_prime

# Forward pass through the merged convolutional layer
merged_output = merged_conv(x)

# Verify the outputs are close
assert torch.allclose(original_output, merged_output,
                      atol=1e-6), "The outputs are not close enough!"

print("Merging was successful!")

RuntimeError: The size of tensor a (16384) must match the size of tensor b (16) at non-singleton dimension 0

In [5]:
import torch
import torch.nn as nn

# Define the convolutional layer and layer normalization
conv = nn.Conv2d(in_channels=3, out_channels=16,
                 kernel_size=3, stride=1, padding=1)
layer_norm = nn.LayerNorm([16, 32, 32])

# Example input
x = torch.randn(1, 3, 32, 32)

# Forward pass through the original layers
conv_output = conv(x)
original_output = layer_norm(conv_output)

# Extract parameters
W = conv.weight.data
b = conv.bias.data
gamma = layer_norm.weight.data
beta = layer_norm.bias.data
epsilon = layer_norm.eps

# Compute mean and variance of the convolutional output
mu = torch.mean(conv_output, dim=(0, 2, 3), keepdim=True)
sigma2 = torch.var(conv_output, dim=(0, 2, 3), keepdim=True, unbiased=False)

# Compute new weights and biases
W_prime = (gamma.view(-1, 1, 1, 1) * W) / torch.sqrt(sigma2 + epsilon)
b_prime = beta + gamma * (b - mu.squeeze()) / \
    torch.sqrt(sigma2.squeeze() + epsilon)

# Create a new convolutional layer with the merged parameters
merged_conv = nn.Conv2d(in_channels=3, out_channels=16,
                        kernel_size=3, stride=1, padding=1)
merged_conv.weight.data = W_prime
merged_conv.bias.data = b_prime

# Forward pass through the merged convolutional layer
merged_output = merged_conv(x)

# Verify the outputs are close
assert torch.allclose(original_output, merged_output,
                      atol=1e-6), "The outputs are not close enough!"

print("Merging was successful!")

RuntimeError: The size of tensor a (16384) must match the size of tensor b (16) at non-singleton dimension 0